
# UBER Pickups 

* Have a map with hot-zones
* Compare results with two unsupervised algorithms. 
* Hot-zones per day of week. 

In [1]:
!pip install plotly==4.9.0 -q
!pip install jupyterlab "ipywidgets>=7.5" -q
!jupyter labextension install jupyterlab-plotly@4.9.0 -q
!jupyter labextension install @jupyter-widgets/jupyterlab-manager plotlywidget@4.9.0 -q

usage: jupyter-labextension [-h] [--debug] [--generate-config] [-y]
                            [--no-build] [--clean]
                            [--log-level InstallLabExtensionApp.log_level]
                            [--config InstallLabExtensionApp.config_file]
                            [--app-dir InstallLabExtensionApp.app_dir]
                            [--dev-build InstallLabExtensionApp.dev_build]
                            [--minimize InstallLabExtensionApp.minimize]
                            [--debug-log-path InstallLabExtensionApp.debug_log_path]
                            [--pin-version-as InstallLabExtensionApp.pin]
                            [extra_args [extra_args ...]]
jupyter-labextension: error: argument -q: expected one argument
usage: jupyter-labextension [-h] [--debug] [--generate-config] [-y]
                            [--no-build] [--clean]
                            [--log-level InstallLabExtensionApp.log_level]
                            [--config 

In [2]:
!pip install plotly
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_curve
#Clustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


import warnings


warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings



import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
# setting Jedha color palette as default
pio.templates["jedha"] = go.layout.Template(
    layout_colorway=["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
)
pio.templates.default = "jedha"
pio.renderers.default = "iframe_connected" # to be replaced by "iframe" if working on JULIE

In [3]:
uber = pd.read_csv("data_uber/uber-raw-data-jun14.csv")
zone = pd.read_csv("data_uber/taxi-zone-lookup.csv")

In [4]:
uber = uber.sample(50000)

In [5]:
uber.head()

,Date/Time,Lat,Lon,Base
545828,6/13/2014 11:07:00,40.7200,-73.9880,B02682
151050,6/14/2014 17:48:00,40.7680,-73.9605,B02598
545891,6/13/2014 11:18:00,40.7858,-73.9510,B02682
553689,6/14/2014 10:06:00,40.7189,-73.9851,B02682
167844,6/17/2014 10:43:00,40.7518,-73.9863,B02598


In [6]:
uber.drop('Base', axis = 1, inplace = True) #We won't use the base column

In [7]:
uber.describe(include = 'all')

,Date/Time,Lat,Lon
count,50000,50000.000000,50000.000000
unique,26360,NaN,NaN
top,6/25/2014 18:45:00,NaN,NaN
freq,11,NaN,NaN
mean,NaN,40.739945,-73.973899
std,NaN,0.038170,0.056795
min,NaN,40.093700,-74.657900
25%,NaN,40.721800,-73.996600
50%,NaN,40.743800,-73.983500
75%,NaN,40.761300,-73.967100


In [8]:
zone.head() #Just to see what it looks like and if we could use it

,LocationID,Borough,Zone
0,1,EWR,Newark Airport
1,2,Queens,Jamaica Bay
2,3,Bronx,Allerton/Pelham Gardens
3,4,Manhattan,Alphabet City
4,5,Staten Island,Arden Heights


In [9]:
uber['Date/Time'] = pd.to_datetime(uber['Date/Time'], format = '%m/%d/%Y %H:%M:%S')

In [10]:
uber['hour'] = uber['Date/Time'].dt.hour
#uber['dayofweek'] = uber['Date/Time'].dt.dayofweek we'll add this line at the end to animate our map by days of the week

In [11]:
new_uber = uber.drop('Date/Time', axis = 1)

In [12]:
new_uber.head()

,Lat,Lon,hour
545828,40.7200,-73.9880,11
151050,40.7680,-73.9605,17
545891,40.7858,-73.9510,11
553689,40.7189,-73.9851,10
167844,40.7518,-73.9863,10


In [13]:
#Standardizing
sc = StandardScaler() #As we only have numeric features 
X = sc.fit_transform(new_uber)

### Clustering with Kmeans

In [14]:

# Instanciate KMeans with k=3 and initialisation with k-means++
# You should always use k-means++ as it alleviate the problem of local minimum convergence 
kmeans = KMeans(n_clusters=3, random_state=0, init="k-means++")

# Fit kmeans to our dataset
kmeans.fit(X)

print('Fitting Kmeans')

Fitting Kmeans


In [15]:
# Let's create a loop that will collect the Within-sum-of-square (wcss) for each value K 
# Let's use .inertia_ parameter to get the within sum of square value for each value K 
wcss =  []
k = []
for i in range (1,11): 
    kmeans = KMeans(n_clusters= i, init = "k-means++", random_state = 0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    k.append(i)
    print("WCSS for K={} --> {}".format(i, wcss[-1]))
    

WCSS for K=1 --> 149999.99999999936
WCSS for K=2 --> 113759.61727042256
WCSS for K=3 --> 84527.80251926028
WCSS for K=4 --> 69289.23474982542
WCSS for K=5 --> 60683.34494248733
WCSS for K=6 --> 53183.10812605771
WCSS for K=7 --> 46964.40974846833
WCSS for K=8 --> 42858.253085172735
WCSS for K=9 --> 39223.530239359454
WCSS for K=10 --> 35322.35854730199


In [16]:
# Create DataFrame
wcss_frame = pd.DataFrame(wcss)
k_frame = pd.Series(k)

# Create figure
fig= px.line(
    wcss_frame,
    x=k_frame,
    y=wcss_frame.iloc[:,-1]
)

# Create title and axis labels
fig.update_layout(
    yaxis_title="Inertia",
    xaxis_title="# Clusters",
    title="Inertia per cluster"
)

# Render
#fig.show(renderer="notebook")
fig.show(renderer="iframe") # if using workspace

In [17]:


# Computer mean silhouette score
sil = []
k = []

## Careful, you need to start at i=2 as silhouette score cannot accept less than 2 labels 
for i in range (2,7): 
    kmeans = KMeans(n_clusters= i, init = "k-means++")
    kmeans.fit(X)
    sil.append(silhouette_score(X, kmeans.predict(X)))
    k.append(i)
    print("Silhouette score for K={} is {}".format(i, sil[-1]))

Silhouette score for K=2 is 0.33343185293884936
Silhouette score for K=3 is 0.3729840291660066
Silhouette score for K=4 is 0.3061658647800318
Silhouette score for K=5 is 0.3459356061634207
Silhouette score for K=6 is 0.3112776016405233


In [18]:
# Create a data frame 
cluster_scores=pd.DataFrame(sil)
k_frame = pd.Series(k)

# Create figure
fig = px.bar(data_frame=cluster_scores,  
             x=k, 
             y=cluster_scores.iloc[:, -1]
            )

# Add title and axis labels
fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="# Clusters",
    title="Silhouette Score per cluster"
)

# Render
#fig.show(renderer="notebook")
fig.show(renderer="iframe") # if using workspace

In [19]:
#We could keep only 3 clusters as the silouhette score is the highest in this case.

### Trying dbScan

In [20]:
X2 = sc.fit_transform(new_uber)

In [21]:
# Instanciate DBSCAN 
db = DBSCAN(eps=1.2, min_samples=60, metric="manhattan", algorithm="brute") 

# Fit on data 
## No need to normalize data, it already is! 
db.fit(X2)

DBSCAN(algorithm='brute', eps=1.2, metric='manhattan', min_samples=60)

In [22]:
np.unique(db.labels_) #We get almost the same numbers of cluster than we've got with Kmeans, It looks pretty fine

array([-1,  0,  1])

In [23]:
new_uber.loc[:,'dbcluster'] = db.labels_
new_uber

,Lat,Lon,hour,dbcluster
545828,40.7200,-73.9880,11,0
151050,40.7680,-73.9605,17,0
545891,40.7858,-73.9510,11,0
553689,40.7189,-73.9851,10,0
167844,40.7518,-73.9863,10,0
...,...,...,...,...
205475,40.7798,-73.9743,16,0
56758,40.7457,-73.9530,11,0
229780,40.6837,-73.9322,9,0
72999,40.7215,-73.9949,21,0


### Visualizing our clusters

In [24]:
fig1 = px.scatter_mapbox(new_uber, lat="Lat", lon="Lon", color="dbcluster",zoom=9)
fig1.update_layout(mapbox_style="open-street-map")
fig1

### Getting number of demands by zones (to highlight hot zones)

In [25]:
#This follow codes will group latitudes and longitudes by zones (Many of the different locations have the same lat and lon until 2 decimals)
new_uber['Lat_rounded'] = new_uber['Lat'].round(2)
new_uber['Lon_rounded'] = new_uber['Lon'].round(2)
new_uber.head()

,Lat,Lon,hour,dbcluster,Lat_rounded,Lon_rounded
545828,40.7200,-73.9880,11,0,40.72,-73.99
151050,40.7680,-73.9605,17,0,40.77,-73.96
545891,40.7858,-73.9510,11,0,40.79,-73.95
553689,40.7189,-73.9851,10,0,40.72,-73.99
167844,40.7518,-73.9863,10,0,40.75,-73.99


In [26]:
#Now let's calculate the number of demands by zone, by day of week
new_uber['dayofweek'] = uber['Date/Time'].dt.dayofweek #We'll add the 'day of week' column now.
new_uber

,Lat,Lon,hour,dbcluster,Lat_rounded,Lon_rounded,dayofweek
545828,40.7200,-73.9880,11,0,40.72,-73.99,4
151050,40.7680,-73.9605,17,0,40.77,-73.96,5
545891,40.7858,-73.9510,11,0,40.79,-73.95,4
553689,40.7189,-73.9851,10,0,40.72,-73.99,5
167844,40.7518,-73.9863,10,0,40.75,-73.99,1
...,...,...,...,...,...,...,...
205475,40.7798,-73.9743,16,0,40.78,-73.97,5
56758,40.7457,-73.9530,11,0,40.75,-73.95,2
229780,40.6837,-73.9322,9,0,40.68,-73.93,2
72999,40.7215,-73.9949,21,0,40.72,-73.99,3


In [27]:
#calculate number of demands
new_uber['demands'] = "" #Empty column to count demands
density = new_uber.groupby(['dayofweek','Lat_rounded','Lon_rounded']).count().sort_values(by = 'dayofweek', ascending = True)
density #We can notice every count here represent the number of demands by dayofweek at different latitudes and longitudes.

Lat  Lon  hour  dbcluster  demands
dayofweek Lat_rounded Lon_rounded                                    
0         40.09       -74.04         1    1     1          1        1
          40.76       -73.90         1    1     1          1        1
                      -73.91         1    1     1          1        1
                      -73.92         9    9     9          9        9
                      -73.93         3    3     3          3        3
...                                ...  ...   ...        ...      ...
6         40.70       -73.99        36   36    36         36       36
                      -74.00        10   10    10         10       10
                      -74.01        29   29    29         29       29
                      -74.07         1    1     1          1        1
          41.12       -73.85         1    1     1          1        1

[2750 rows x 5 columns]

In [28]:
#Transforming indexes into columns
density.reset_index(level=['dayofweek', 'Lat_rounded', 'Lon_rounded'], inplace = True) #Multiindexes as columns
density

,dayofweek,Lat_rounded,Lon_rounded,Lat,Lon,hour,dbcluster,demands
0,0,40.09,-74.04,1,1,1,1,1
1,0,40.76,-73.90,1,1,1,1,1
2,0,40.76,-73.91,1,1,1,1,1
3,0,40.76,-73.92,9,9,9,9,9
4,0,40.76,-73.93,3,3,3,3,3
...,...,...,...,...,...,...,...,...
2745,6,40.70,-73.99,36,36,36,36,36
2746,6,40.70,-74.00,10,10,10,10,10
2747,6,40.70,-74.01,29,29,29,29,29
2748,6,40.70,-74.07,1,1,1,1,1


### Let's re-create our cluster now and animate our map with the number of orders by 'Day of week' (We could do it for any other parameter like day of month, or even hours)

In [29]:
uber_density = density.loc[:,['dayofweek','Lat_rounded','Lon_rounded','demands']]
uber_density

,dayofweek,Lat_rounded,Lon_rounded,demands
0,0,40.09,-74.04,1
1,0,40.76,-73.90,1
2,0,40.76,-73.91,1
3,0,40.76,-73.92,9
4,0,40.76,-73.93,3
...,...,...,...,...
2745,6,40.70,-73.99,36
2746,6,40.70,-74.00,10
2747,6,40.70,-74.01,29
2748,6,40.70,-74.07,1


In [30]:
#Standardize
X3 = sc.fit_transform(uber_density)

#Instanciate DBSCAN 
db = DBSCAN(eps=1.2, min_samples=5, metric="manhattan", algorithm="brute") #These parameters where chosen arbitrary to have more than 2 cluster

# Fit on data 
db.fit(X3)

np.unique(db.labels_)

array([-1,  0,  1])

In [31]:
#Add cluster column to the dataset
uber_density.loc[:,'dbcluster'] = db.labels_
uber_density

,dayofweek,Lat_rounded,Lon_rounded,demands,dbcluster
0,0,40.09,-74.04,1,-1
1,0,40.76,-73.90,1,0
2,0,40.76,-73.91,1,0
3,0,40.76,-73.92,9,0
4,0,40.76,-73.93,3,0
...,...,...,...,...,...
2745,6,40.70,-73.99,36,0
2746,6,40.70,-74.00,10,0
2747,6,40.70,-74.01,29,0
2748,6,40.70,-74.07,1,0


### Visualizing clusters, by days of week, with number of demands.

In [32]:
fig1 = px.scatter_mapbox(uber_density, lat="Lat_rounded", lon="Lon_rounded", 
                         color="dbcluster",
                         size = 'demands',
                         animation_frame= 'dayofweek',
                         zoom=9)
fig1.update_layout(mapbox_style="open-street-map")
fig1